In [ ]:
#Necessary Libraries

import numpy as np
import tensorflow as tf
from keras.utils import np_utils
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Conv2D, BatchNormalization, Dropout, MaxPooling2D, Activation


In [ ]:
!pip install wandb -qqq
import wandb
from wandb.keras import WandbCallback

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
zip_path = "/content/drive/MyDrive/Dataset /nature_12K.zip"
!cp "{zip_path}" .
!unzip -q nature_12K.zip
!rm nature_12K.zip

In [ ]:
# Storing the training and testing directories
import os 
Tags = ['Amphibia','Animalia','Arachnida','Aves','Fungi','Insecta','Mammalia','Mollusca','Plantae','Reptilia']
no_class = 10
train_dir='inaturalist_12K/train/'
test_dir='inaturalist_12K/val/'

In [ ]:
#Generating test data set 
def test_dataset(batch_size=256,augment_data=False):

    if augment_data:
        train_datagen = ImageDataGenerator(rescale=1./255,
                                          rotation_range=45,
                                          zoom_range=0.2,
                                          shear_range=0.2,
                                          horizontal_flip=True,
                                          validation_split=0.1)
        test_datagen = ImageDataGenerator(rescale=1./255)

    else:
        train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1)
        test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(train_dir, target_size=(128, 128),batch_size=batch_size,
                                                        class_mode="categorical",subset='training', seed=1234)
    val_generator = train_datagen.flow_from_directory(train_dir, target_size=(128, 128),batch_size=batch_size,
                                                        class_mode="categorical",subset='validation', seed=1234)
    test_generator = test_datagen.flow_from_directory(test_dir, target_size=(128, 128), batch_size=batch_size,
                                                        class_mode="categorical",seed=1234)
    
    return train_generator,val_generator, test_generator

In [ ]:
#Build a small CNN model consisting of 5 convolution layers.

def CNN(filters,filter_size, image_size=256,
              dropout=0.2,batch_norm=False, dense_size=64, 
              regpara=0, no_of_classes=10, activation='relu'):

    model = Sequential()
    for i in range(5):
        if(i==0):
            model.add(Conv2D(filters=filters[i], kernel_size=filter_size[i], padding = 'same', input_shape = (image_size, image_size, 3),
                             kernel_regularizer= regularizers.l2(regpara)))
        else:
            model.add(Conv2D(filters=filters[i], kernel_size=filter_size[i], padding = 'same',
                             kernel_regularizer= regularizers.l2(regpara)))
        model.add(Activation(activation))
        if batch_norm:
            model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2,2)))

    
    # FC layer
    model.add(Flatten())
    model.add(Dense(dense_size, activation="relu",kernel_regularizer= regularizers.l2(regpara)))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())

    #Output Layer
    model.add(Dense(no_of_classes, activation = "softmax"))

    return model

In [ ]:
#Testing the model with the best configuration on test data
def test():
    
    #best_configuration obtained from training and validation
        best_kernel_size= [(3,3),(3,3),(3,3),(3,3),(3,3)]
        best_weight_decay= 0
        best_dropout= 0.4
        best_learning_rate= 1e-3
        best_activation= 'selu'
        best_batch_size= 128
        best_batch_norm= True
        best_filters= [32,32,32,32,32]
        best_data_augment= False
        best_dense_size= 64
   
        model=CNN(filters=best_filters,filter_size=best_kernel_size, image_size=128,
                  dropout=best_dropout,batch_norm=best_batch_norm, dense_size=best_dense_size, 
                  regpara=best_weight_decay, no_of_classes=10, activation=best_activation)
    
        model.compile(loss = "categorical_crossentropy", optimizer = 'adam', metrics=['accuracy'])
        train_data,val_data,test_data = test_dataset(batch_size=best_batch_size,augment_data=best_data_augment)
        model.fit(train_data, epochs=10,validation_data=val_data)
        model.evaluate(test_data, batch_size = best_batch_size)
        model.save("model-best.h5")

        return model
    

In [ ]:
model=test()

In [ ]:
#taking 30 sample images,3 from each class for visualizing

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2 
images=[]
tag=[]
tag_pred=[]
for tg in Tags:
    i=0
    dir=os.path.join(test_dir,tg)
    for img in os.listdir(dir):
        if i<3:
            image = mpimg.imread(os.path.join(dir,img))
            images.append(image)
            tag.append(tg)
            img_req= cv2.resize(image, (128,128)) / 255.0
            prediction = model.predict(img_req.reshape(1,128, 128,3))           #To predict the given image
            p=prediction.argmax()                                               #outputs an integer value from 0 to 9
            tag_pred.append(Tags[p])
            i+=1

#Plotting a 10x3 grid with true labels and predicted labels
fig = plt.figure(figsize=(10,30))
for i in range(30):
  img=cv2.resize(images[i],(128,128))
  fig.add_subplot(10,3,i+1)
  plt.imshow(img)
  plt.axis('off')
  plt.title('True:'+ tag[i] + ',' + 'Predicted:'+ tag_pred[i],fontdict={'fontsize':10}) 
wandb.init(entity='nomads',project='CS6910_DL_Assignment2')
wandb.log({'Plotting a 10x3 grid with true labels and predicted labels':plt})

In [ ]:
model.summary()

In [ ]:
#Visualising all the filters in the first layer of the best model for a random image from the test set.
#Reference: https://machinelearningmastery.com/how-to-visualize-filters-and-feature-maps-in-convolutional-neural-networks/

#Input is the image to the model 
#Output is the output from the first convolution layer
filter_layer_1 = tf.keras.models.Model(
      inputs = model.inputs,                        
      outputs = model.get_layer('conv2d_5').output)
tag = Tags[2]                                        #To plot the filters for class 2 i.e. 'Arachnida'
dir=os.path.join(test_dir,tag)
image_name=os.listdir(dir)[0]                        #Loading the test data
image_path=(os.path.join(dir,image_name))
image=mpimg.imread(image_path)
img_req=cv2.resize(image,(128,128))
idx = np.expand_dims(img_req, axis=0)                #Expand the shape of the Image array
inputs = tf.cast(idx, tf.float32)                    #To cast a tensor into float type
out = filter_layer_1(inputs)[0]                      

#Plotting the 32 filters in 8x4 grid
no_of_filters= 32
plt.figure(figsize=(20,20))
for i in range(no_of_filters):
    f = out[:, :, i]
    # plot each channel separately
    for j in range(3):
        # specify subplot and turn of axis
        ax = plt.subplot(8, 4, i+1)
        ax.set_xticks([])
        ax.set_yticks([])
        plt.imshow(f)
# show the figure
#plt.show()
wandb.init(entity='nomads',project='CS6910_DL_Assignment2')
wandb.log({'Visualising all the filters in the first layer':plt})